In [1]:
'''Go through movie->features pipeline manually'''
# First load the stimulus
from featurex.tests.utils import get_test_data_path
from os.path import join
from featurex.stimuli.video import VideoStim
from featurex.converters.video import VideoToAudioConverter, FrameSamplingConverter
from featurex.converters.image import TesseractConverter
from featurex.converters.api import WitTranscriptionConverter
from featurex.extractors import ExtractorResult
from featurex.extractors.image import VibranceExtractor
from featurex.extractors.text import LengthExtractor
from featurex.graph import Graph, Node

filename = join(get_test_data_path(), 'video', 'obama_speech.mp4')
video = VideoStim(filename)

In [2]:
# Extract the audio
conv = VideoToAudioConverter()
audio = conv.convert(video)

[MoviePy] Writing audio in /Users/quinnmac/Library/Python/2.7/lib/python/site-packages/featurex-0.0.1-py2.7.egg/featurex/tests/data/video/obama_speech.wav


100%|██████████| 193/193 [00:00<00:00, 1654.85it/s]

[MoviePy] Done.


In [3]:
# Subsample the video
conv = FrameSamplingConverter(every=15)
derived = conv.convert(video)

In [4]:
# Extract text from the video
conv = TesseractConverter()
visual_texts = []
for frame in derived:
    visual_texts.append(conv.convert(frame))

In [5]:
# Extract text from the audio
conv = WitTranscriptionConverter()
audio_text = conv.convert(audio)

In [6]:
# Extract a low-level image feature from each frame
ext = VibranceExtractor()
visual_features = []
for frame in derived:
    visual_features.append(ext.extract(frame))

In [7]:
# Extract word length from both audio and visual text
ext = LengthExtractor()
visual_length = [ext.extract(t) for t in visual_texts]
audio_length = [ext.extract(t) for t in audio_text]

In [8]:
# Merge and display results
print ExtractorResult.merge_stims(visual_features)
print ExtractorResult.merge_stims(visual_length)
print ExtractorResult.merge_stims(audio_length)

                       onset  duration    vibrance
  stim                                            
0 obama_speech.mp4_0    0.00       NaN  418.851528
  obama_speech.mp4_15   1.25       NaN  441.618626
  obama_speech.mp4_30   2.50       NaN  506.321013
  obama_speech.mp4_45   3.75       NaN  575.903070
  obama_speech.mp4_60   5.00       NaN  586.500651
  obama_speech.mp4_75   6.25       NaN  590.007483
  obama_speech.mp4_90   7.50       NaN  589.524823
                                                      onset  duration  \
  stim                                                                  
0 obama_speech.mp4_0_                                  0.00       NaN   
  obama_speech.mp4_15_                                 1.25       NaN   
  obama_speech.mp4_30_                                 2.50       NaN   
  obama_speech.mp4_45_mslnsu-r onu‘ﬁ. SAVEIENV nu...   3.75       NaN   
  obama_speech.mp4_60_1'"\nPIESIIIE Y' I' I ‘ s\n...   5.00       NaN   
  obama_speech.mp4_75_        

In [9]:
# Graph way of doing the same pipeline
visual_nodes = [(FrameSamplingConverter(every=15), 'framesampling', 
             [(TesseractConverter(), 'visual_text', 
                [(LengthExtractor(), 'text_length')]), 
              (VibranceExtractor(), 'visual_vibrance')])]
audio_nodes = [(VideoToAudioConverter(), 'audio', 
            [(WitTranscriptionConverter(), 'audio_text', 
              [(LengthExtractor(), 'text_length')])])]

graph = Graph()
graph.add_children(visual_nodes)
graph.add_children(audio_nodes)
graph.extract(video)

[MoviePy] Writing audio in /Users/quinnmac/Library/Python/2.7/lib/python/site-packages/featurex-0.0.1-py2.7.egg/featurex/tests/data/video/obama_speech.wav


100%|██████████| 193/193 [00:00<00:00, 1710.80it/s]

[MoviePy] Done.


LengthExtractor  \
                                                         text_length   
  stim                                                                 
0 obama_speech.mp4_60                                            NaN   
  obama_speech.mp4_obama_speech.wav_today                          5   
  obama_speech.mp4_obama_speech.wav_together                       8   
  obama_speech.mp4_60_1'"\nPIESIIIE Y' I' I ‘ s\n...              56   
  obama_speech.mp4_obama_speech.wav_united                         6   
  obama_speech.mp4_obama_speech.wav_cheats                         6   
  obama_speech.mp4_obama_speech.wav_the                            3   
  obama_speech.mp4_obama_speech.wav_who                            3   
  obama_speech.mp4_45                                            NaN   
  obama_speech.mp4_45_mslnsu-r onu‘ﬁ. SAVEIENV nu...              36   
  obama_speech.mp4_obama_speech.wav_or                             2   
  obama_speech.mp4_15                                            NaN   
  obama_speech.mp4_30_                                             0   
  obama_speech.mp4_obama_speech.wav_negotiations                  12   
  obama_speech.mp4_30                                            NaN   
  obama_speech.mp4_75_                                             0   
  obama_speech.mp4_obama_speech.wav_two                            3   
  obama_speech.mp4_15_                                             0   
  obama_speech.mp4_obama_speech.wav_after                          5   
  obama_speech.mp4_0                                             NaN   
  obama_speech.mp4_obama_speech.wav_years                          5   
  obama_speech.mp4_obama_speech.wav_partners                       8   
  obama_speech.mp4_obama_speech.wav_states                         6   
  obama_speech.mp4_obama_speech.wav_international                 13   
  obama_speech.mp4_0_                                              0   
  obama_speech.mp4_90                                            NaN   
  obama_speech.mp4_obama_speech.wav_of                             2   
  obama_speech.mp4_75                                            NaN   
  obama_speech.mp4_90_, 7 V\nPnEslnEMJ'W min: an ...              36   

                                                     VibranceExtractor  onset  
                                                              vibrance         
  stim                                                                         
0 obama_speech.mp4_60                                       586.500651   5.00  
  obama_speech.mp4_obama_speech.wav_today                          NaN   0.00  
  obama_speech.mp4_obama_speech.wav_together                       NaN   9.00  
  obama_speech.mp4_60_1'"\nPIESIIIE Y' I' I ‘ s\n...               NaN   5.00  
  obama_speech.mp4_obama_speech.wav_united                         NaN   7.00  
  obama_speech.mp4_obama_speech.wav_cheats                         NaN  14.00  
  obama_speech.mp4_obama_speech.wav_the                            NaN   6.00  
  obama_speech.mp4_obama_speech.wav_who                            NaN  13.00  
  obama_speech.mp4_45                                       575.903070   3.75  
  obama_speech.mp4_45_mslnsu-r onu‘ﬁ. SAVEIENV nu...               NaN   3.75  
  obama_speech.mp4_obama_speech.wav_or                             NaN  10.00  
  obama_speech.mp4_15                                       441.618626   1.25  
  obama_speech.mp4_30_                                             NaN   2.50  
  obama_speech.mp4_obama_speech.wav_negotiations                   NaN   5.00  
  obama_speech.mp4_30                                       506.321013   2.50  
  obama_speech.mp4_75_                                             NaN   6.25  
  obama_speech.mp4_obama_speech.wav_two                            NaN   2.00  
  obama_speech.mp4_15_                                             NaN   1.25  
  obama_speech.mp4_obama_speech.wav_after                          N